## Initialization

In [ ]:

#! STEP1: In terminal:
#! source Thesis/bin/activate

In [1]:
import mne
import pandas as pd
import numpy as np
import os
import matplotlib
from mne.filter import filter_data
from mne.preprocessing import ICA
from autoreject import AutoReject
matplotlib.use('TkAgg')

## File selection/Filtering

In [2]:
All_data = []
All_resting = []
All_audio = []
All_video = []

Migraine_resting = []
Migraine_audio = []
Migraine_video = []

Control_resting = []
Control_audio = []
Control_video = []

In [3]:
# Sorting all the file names into their respective categories
for File in os.listdir('Dataset'):
    if ".bdf" in File: # only looking at the EEG files
        All_data.append(File)
        if "Resting" in File: # selecting only resting-state files
            All_resting.append(File)
            if File[0] == "M": # selecting migraineurs among resting-state files
                Migraine_resting.append(File)
            elif File[0] == "C": # selecting control group among resting-state files
                Control_resting.append(File)
        elif "SSAEP" in File: # selecting only audio stimulation
            All_audio.append(File)
            if File[0] == "M": # selecting migraineurs among audio stimulation files
                Migraine_audio.append(File)
            elif File[0] == "C": # selecting control group among audio stimulation files
                Control_audio.append(File)
        elif "SSVEP" in File: # selecting only video stimulation
            All_video.append(File)
            if File[0] == "M": # selecting migraineurs among video stimulation files
                Migraine_video.append(File)
            elif File[0] == "C": # selecting control group among video stimulation files
                Control_video.append(File)

In [4]:

#? To be used when turning lists into dataframes (if needed)
All = {"Resting": All_resting, "Audio": All_audio, "Video": All_video}
Migraines = {"Resting": Migraine_resting, "Audio": Migraine_audio, "Video": Migraine_video}
Controls = {"Resting": Control_resting, "Audio": Control_audio, "Video": Control_video}

In [5]:
# File integrity inspection
print(len(All_data), len(All_resting), len(All_audio), len(All_video))

print(len(Migraine_resting), len(Migraine_audio), len(Migraine_video))
print(Migraine_audio)
#! Migraine patient 13 has no audio file

print(len(Control_resting), len(Control_audio), len(Control_video))
print(Control_audio)
#! Control patient 14 has two audiofiles, maybe due to an interruption in the recording? --> documentation shows no info about this

117 39 39 39
17 16 17
['M10_SSAEP.bdf', 'M11_SSAEP.bdf', 'M12_SSAEP.bdf', 'M14_SSAEP.bdf', 'M16_SSAEP.bdf', 'M17_SSAEP.bdf', 'M18_SSAEP.bdf', 'M1_SSAEP.bdf', 'M2_SSAEP.bdf', 'M3_SSAEP.bdf', 'M4_SSAEP.bdf', 'M5_SSAEP.bdf', 'M6_SSAEP.bdf', 'M7_SSAEP.bdf', 'M8_SSAEP.bdf', 'M9_SSAEP.bdf']
21 22 21
['C10_SSAEP.bdf', 'C11_SSAEP.bdf', 'C12_SSAEP.bdf', 'C13_SSAEP.bdf', 'C14_SSAEP.bdf', 'C14_SSAEP_2.bdf', 'C15_SSAEP.bdf', 'C16_SSAEP.bdf', 'C17_SSAEP.bdf', 'C18_SSAEP.bdf', 'C19_SSAEP.bdf', 'C1_SSAEP.bdf', 'C20_SSAEP.bdf', 'C21_SSAEP.bdf', 'C2_SSAEP.bdf', 'C3_SSAEP.bdf', 'C4_SSAEP.bdf', 'C5_SSAEP.bdf', 'C6_SSAEP.bdf', 'C7_SSAEP.bdf', 'C8_SSAEP.bdf', 'C9_SSAEP.bdf']


In [6]:
# Preparing the event ID's as per the documentation (EEG experiment protocol_Migraine_Brain_KiltHub_vF.pdf)
resting_event_id = {"block_start": 1, "block_end": 2}

stimulation_event_id = {"4hz_stim_start": 1, "4hz_stim_keypress":11, "6hz_stim_start": 2, "6hz_stim_keypress": 22}

## Temp Code

In [ ]:
# Inspecting the data for bad channels or artifacts
"""
for File in All_resting:
    print("Now looking at: " + File)
    raw = mne.io.read_raw_bdf(f"Dataset/{File}")
    events = mne.find_events(raw)
    fig = mne.viz.plot_events(events, sfreq=raw.info["sfreq"], first_samp=raw.first_samp, event_id=resting_event_id)
    epochs = mne.Epochs(raw, events)
    raw.plot()
    epochs['1'].plot(events=True)

for File in All_audio:
    print("Now looking at: " + File)
    raw = mne.io.read_raw_bdf(f"Dataset/{File}")
    events = mne.find_events(raw)
    fig = mne.viz.plot_events(events, sfreq=raw.info["sfreq"], first_samp=raw.first_samp, event_id=stimulation_event_id)
    epochs = mne.Epochs(raw, events)
    raw.plot()
    epochs['1'].plot(events=True)

for File in All_video:
    print("Now looking at: " + File)
    raw = mne.io.read_raw_bdf(f"Dataset/{File}")
    events = mne.find_events(raw)
    fig = mne.viz.plot_events(events, sfreq=raw.info["sfreq"], first_samp=raw.first_samp, event_id=stimulation_event_id)
    epochs = mne.Epochs(raw, events)
    raw.plot()
    epochs['1'].plot(events=True)
"""

## Evaluation Matrices

In [ ]:
#Evalution matrices
"""
## LOSS-ACC GRAPHS
def loss_acc_graph(MODEL, modelname):
    plt.figure()
    plt.subplot(211)
    plt.plot(MODEL.history['loss'], label="Loss")
    plt.plot(MODEL.history['val_loss'], label="Validation Loss")
    plt.title (modelname + " Training/Validation Loss and Accuracy")
    plt.legend()

    plt.subplot(212)
    plt.plot(MODEL.history['accuracy'], label="Accuracy")
    plt.plot(MODEL.history['val_accuracy'], label="Validation Accuracy")
    plt.legend()
    plt.show()
    
#Conf matrix and Classification Report:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

def calculate_metrics(model, X_test, y_test, title):
    # predict probabilities for test set
    y_pred = model.predict(X_test, verbose=0)
    
    # ensuring same format for both
    y_pred=np.argmax(y_pred, axis=1)
    y_test=np.argmax(y_test, axis=1)
    
    # using classification report to get metrics
    metrics = classification_report(y_test, y_pred, target_names=ordered_classes)
    print(metrics)
    
    # confusion matrix
    matrix = confusion_matrix(y_test, y_pred)
    p = sns.heatmap(matrix, annot=True, fmt='g')
    p.set_xlabel("Predicted label", fontsize = 20)
    p.set_ylabel("True Label", fontsize = 20)
    p.xaxis.set_ticklabels(["CBT", "CA", "LBT", "LQCC", "LA"], rotation = 0)
    p.yaxis.set_ticklabels(ordered_classes, rotation = 0)
    plt.title (title)
    plt.show()
    
#ROC curves
from sklearn.metrics import roc_curve, RocCurveDisplay, auc, roc_auc_score

def roc(model, X_test, y_test,title):
      # predict probabilities for test set
      y_pred = model.predict(X_test, verbose=0)

      # initializing the plot
      plt.figure()
      plt.plot([0, 1], [0, 1], 'k--')
      plt.title (title)
      plt.xlabel('False Positive Rate')
      plt.ylabel('True Positive Rate')
      
      
      # computing micro and macro AUC scores #TODO: does not correctly plot micro and macro AUC
      micro_auc = roc_auc_score(y_test, y_pred, average="micro", multi_class="ovr")
      macro_auc = roc_auc_score(y_test, y_pred, average="macro", multi_class="ovr")
      plt.plot(micro_auc, label=f'ROC (micro) - area = {micro_auc:.3f}')
      plt.plot(macro_auc, label=f'ROC (macro) - area = {macro_auc:.3f}')
      
      # computing AUC scores per class and plotting them
      for i in range(5):
            fpr, tpr, threshold = roc_curve(y_test[:, i], y_pred[:, i])
            auc_val = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'ROC ({ordered_classes[i]}) - area = {auc_val:.3f}')
      
      plt.legend(loc="lower right")
      plt.show()
"""

## Preprocessing

In [12]:
# For testing purposes
#raw.plot()

<MNEBrowseFigure size 2548x1367 with 4 Axes>

Channels marked as bad:
none


In [ ]:
# Reading in data, finding events, setting up epochs
raw = mne.io.read_raw_bdf(f"Dataset/{All_resting[0]}", preload=True)
events = mne.find_events(raw)
epochs = mne.Epochs(raw, events, preload=True)
raw.set_eeg_reference(ref_channels=['M1', 'M2']) #(i) EEG data were re-referenced offline to the average of the two mastoid electrodes
raw.set_channel_types({'ECG':'ecg', 'Fp1':'eog'}) # setting up pre-existing channel ECG for heartbeat detection; Fp1 as eye-near channel is used as proxy for eye-related artifacts

In [ ]:
raw_filt = raw.copy().filter(l_freq=0.1, h_freq=100) #(i) zero-phase Butterworth filter was used to filter the signals between 0.1 and 100 Hz;
raw_filt.info

In [ ]:
for File in All_resting:
    raw = mne.io.read_raw_bdf(f"Dataset/{File}", preload=True)
    raw.set_eeg_reference(ref_channels=['M1', 'M2'])
    raw.set_channel_types({'ECG':'ecg', 'Fp1':'eog'})
    raw2 = raw.copy()
    raw2.info["bads"] = []
    events = mne.find_events(raw2)
    epochs = mne.Epochs(raw2, events=events)["2"].average().plot()
#TODO (ii) noisy channels were detected visually and interpolated (this was done in 0.95% of electrodes from the migraine group and 0.3% from the control group); 


In [ ]:
# Since data is not supposed to be filtered prior to ICA, a copy is used to conduct the ica. 50 components have been chosen to limit the time it takes (while still explaining ~96% of variation)
ica_filt_raw = raw.copy().filter(l_freq=1.0, h_freq=None)
ica = ICA(n_components=50, max_iter="auto", random_state=97)
ica.fit(ica_filt_raw)
ica

explained_var_ratio = ica.get_explained_variance_ratio(ica_filt_raw)
for channel_type, ratio in explained_var_ratio.items():
    print(
        f"Fraction of {channel_type} variance explained by all components: " f"{ratio}"
    )
    
raw.load_data()
ica.plot_sources(raw, show_scrollbars=False)

In [ ]:
# Finding out which components to exclude
ica.exclude = []
# find which ICs match the EOG/ECG patterns
eog_indices, eog_scores = ica.find_bads_eog(raw)
ica.exclude = eog_indices
ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method="correlation", threshold="auto")
ica.exclude += ecg_indices

# barplot of ICA component "EOG match" scores
ica.plot_scores(eog_scores)
ica.plot_scores(ecg_scores)
# plot ICs applied to raw data, with EOG/ECG matches highlighted
ica.plot_sources(raw, show_scrollbars=False)

In [ ]:
#(iv) for the resting state trials, 2-s non-overlapping time intervals were extracted; and 
#(v)  these 2-s time intervals were passed through zero-phase Kaiser bandpass filters to extract 5 frequency bands of delta (0.5 − 3 Hz), theta (4 − 7 Hz), alpha (8 − 12 Hz), beta (12 − 30 Hz) and gamma (30 − 100 Hz). 
# A lowpass filter with cut-off frequency of 30 Hz is standardly used for noisy EEG data64. 

epoch_length = 2  # Length of each epoch in seconds
sfreq = raw.info['sfreq']  # Sampling frequency of the data

# Extract 2-second non-overlapping epochs
n_samples_per_epoch = int(epoch_length * sfreq)
n_epochs = len(raw.times) // n_samples_per_epoch
epochs_data = np.array_split(raw.get_data(), n_epochs, axis=1)

# Initialize lists to store filtered data for each frequency band
delta_data = []
theta_data = []
alpha_data = []
beta_data = []
gamma_data = []

# Define frequency bands
freq_bands = {
    'delta': (0.5, 3),
    'theta': (4, 7),
    'alpha': (8, 12),
    'beta': (12, 30),
    'gamma': (30, 100)
}

# Apply zero-phase Kaiser bandpass filters to extract frequency bands
for epoch_data in epochs_data:
    for freq_band, (fmin, fmax) in freq_bands.items():
        filtered_data = filter_data(epoch_data, sfreq, fmin, fmax, method='fir', phase='zero')
        if freq_band == 'delta':
            delta_data.append(filtered_data)
        elif freq_band == 'theta':
            theta_data.append(filtered_data)
        elif freq_band == 'alpha':
            alpha_data.append(filtered_data)
        elif freq_band == 'beta':
            beta_data.append(filtered_data)
        elif freq_band == 'gamma':
            gamma_data.append(filtered_data)

# Concatenate the filtered data arrays
delta_data = np.concatenate(delta_data, axis=1)
theta_data = np.concatenate(theta_data, axis=1)
alpha_data = np.concatenate(alpha_data, axis=1)
beta_data = np.concatenate(beta_data, axis=1)
gamma_data = np.concatenate(gamma_data, axis=1)

# Apply lowpass filter with cutoff frequency of 30 Hz
delta_data = filter_data(delta_data, sfreq, None, 30, method='fir', phase='zero')
theta_data = filter_data(theta_data, sfreq, None, 30, method='fir', phase='zero')
alpha_data = filter_data(alpha_data, sfreq, None, 30, method='fir', phase='zero')
beta_data = filter_data(beta_data, sfreq, None, 30, method='fir', phase='zero')
gamma_data = filter_data(gamma_data, sfreq, None, 30, method='fir', phase='zero')

In [79]:
# Shows the event points
fig = mne.viz.plot_events(events, sfreq=raw.info["sfreq"], first_samp=raw.first_samp, event_id=resting_event_id)

In [ ]:

#TODO: Implement pre-processing strategy as written by chamanzar et al:
#(i) EEG data were re-referenced offline to the average of the two mastoid electrodes and a zero-phase Butterworth filter was used to filter the signals between 0.1 and 100 Hz; 
#! Why is this done? --> The mastoids are channels which are mean to pick up all the environmental noise, while not capturing brain activity. Sort of a "blueprint" for what to count as artifacts
"""
for File in All_resting:
    print("Now filtering: " + File)
    raw = mne.io.read_raw_bdf(f"Dataset/{File}", preload=True)
    raw.set_eeg_reference(ref_channels=['M1', 'M2'])
    raw_filtered = filter_data(raw.get_data(), sfreq=raw.info['sfreq'], l_freq=0.1, h_freq=100, method='fir', verbose=False)
    raw._data = raw_filtered
    #raw.plot(block=True)
"""

#(iv) for the visual and auditory trials, the first 2 s after the stimulus onset were extracted, 
#However, we chose to keep the high-frequency gamma band given that: 
#(a) the Biosemi ActiveTwo system has active electrodes and it can tolerate high electrode impedances, 
#(b) the participants were seated inside a Faraday cage during the EEG recording to reduce the electromagnetic interferences and 
#(c) we detected and interpolated noisy channels, as stated in Step (ii) of the preprocessing.

## Data Splitting

In [ ]:

#TODO: set up CV (LOOCV and 5-fold random CV)

## Data Augmentation

In [ ]:

#! Split up the recordings into parts, BUT: Make sure that all parts of one participant end up in the same fold
#! Classifier will evaluate each epoch separately, but could be useful to aggregate them --> if majority of the evaluations classify patient, then it is considered a patient